# AdaBoost

### Nome: Ronald Davi Rodrigues Pereira
### Matrícula: 2019711249

In this project, we will implement the [Adaptative Boosting model (AdaBoost)](https://www.sciencedirect.com/science/article/pii/S002200009791504X) for the [Tic-Tac-Toe UCI Machine Learning Repository dataset](https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame).

This dataset contains all possible Tic-Tac-Toe games possibilities and its outcome for a given player using the 'x' mark. The label of this dataset indicates if 'x' won or lost.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from itertools import product

## Reading the dataset as CSV file

1. top-left-square: {x,o,b} -> 'tl'
2. top-middle-square: {x,o,b} -> 'tm'
3. top-right-square: {x,o,b} -> 'tr'
4. middle-left-square: {x,o,b} -> 'ml'
5. middle-middle-square: {x,o,b} -> 'mm'
6. middle-right-square: {x,o,b} -> 'mr'
7. bottom-left-square: {x,o,b} -> 'bl'
8. bottom-middle-square: {x,o,b} -> 'bm'
9. bottom-right-square: {x,o,b} -> 'br'
10. Class: {positive,negative} -> 'label'

In [2]:
tic_tac_toe_df = pd.read_csv('data/tic-tac-toe.data', header=None)
tic_tac_toe_df.columns = ['tl', 'tm', 'tr', 'ml', 'mm', 'mr', 'bl', 'bm', 'br', 'label']
tic_tac_toe_df.head()

,tl,tm,tr,ml,mm,mr,bl,bm,br,label
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


## Dataset Preprocessing

The objective here is to convert all symbols in the Tic-Tac-Toe dataset to numerical and boolean values.

Features:

- The 'x' is replaced by +1
- The 'o' is replaced by -1
- The 'b' is replaced by 0

Labels:

- The positive label is replaced by +1
- The negative label is replaced by -1

In [3]:
replace_dict = {'x': 1, 'o':-1, 'b':0, 'positive':1, 'negative': -1}
tic_tac_toe_df.replace(to_replace=replace_dict, inplace=True)
tic_tac_toe_df.head()

,tl,tm,tr,ml,mm,mr,bl,bm,br,label
0,1,1,1,1,-1,-1,1,-1,-1,1
1,1,1,1,1,-1,-1,-1,1,-1,1
2,1,1,1,1,-1,-1,-1,-1,1,1
3,1,1,1,1,-1,-1,-1,0,0,1
4,1,1,1,1,-1,-1,0,-1,0,1


## AdaBoost model



In [4]:
class AdaBoost():
    def __init__(self, tic_tac_toe_df:pd.DataFrame):
        self.dataset = tic_tac_toe_df
        self.weights = np.ones(self.dataset.shape[0]) * 1/self.dataset.shape[0]

    def classify_stump(self, stump:tuple, line_n:int):
        feature_value = self.dataset.loc[line_n, stump[0]]
        
        if ((feature_value == replace_dict['x'] and stump[1]) or
            (feature_value == replace_dict['b'] and stump[2]) or
            (feature_value == replace_dict['o'] and stump[3])):
            return stump[4]
        
        return -stump[4]

    def find_best_stump(self):
        best_stump = tuple()
        best_empirical_error = float('inf')

        positions = [position for position in self.dataset.columns[:-1]]
        is_x = [False, True]
        is_b = [False, True]
        is_o = [False, True]
        label = [-1, 1]

        stumps = product(positions, is_x, is_b, is_o, label)

        for stump in stumps:
            current_error = 0
            for i in range(self.dataset.shape[0]):
                if self.classify_stump(stump, i) != self.dataset.loc[i, 'label']:
                    current_error += self.weights[i]
            if current_error < best_empirical_error:
                best_empirical_error = current_error
                best_stump = stump

        try:
            alpha = (1/2) * np.log2((1-best_empirical_error) / best_empirical_error)
        except ZeroDivisionError:
            alpha = 1.0

        return alpha, best_stump

    def update_weights(self, alpha:float, stump:tuple):
        for i in range(self.weights.shape[0]):
            self.weights[i] /= np.sum(self.weights)
            self.weights[i] *= np.exp(-alpha * self.classify_stump(stump, i) * self.dataset.loc[i, 'label'])

    def predict(self):
        predictions = []
        for n in range(self.dataset.shape[0]):
            prediction = 0
            for i in range(len(self.stumps)):
                prediction += self.alphas[i] * self.classify_stump(self.stumps[i], n)
            predictions.append(-1 if prediction < 0 else 1)

        return predictions

    def accuracy_score(self):
        predictions = self.predict()
        correct = 0
        for n in range(self.dataset.shape[0]):
            if predictions[n] == self.dataset.loc[n, 'label']:
                correct += 1
        return correct / self.dataset.shape[0]

    def train(self, n_rounds:int=100, early_stopping_rounds:int=10):
        self.alphas = []
        self.stumps = []
        self.accuracies = []
        best_accuracy = 0
        no_improvement_rounds = 0

        for n in range(n_rounds):
            alpha, stump = self.find_best_stump()
            self.alphas.append(alpha)
            self.stumps.append(stump)

            accuracy = self.accuracy_score()

            self.update_weights(alpha, stump)

            if accuracy <= best_accuracy:
                no_improvement_rounds += 1
            else:
                best_accuracy = accuracy
                no_improvement_rounds = 0

            if no_improvement_rounds >= early_stopping_rounds:
                print('-'*10 + 'Early Stopping limit reached' + '-'*10)
                print(f'[Best Round {n-no_improvement_rounds+1}] Train_Accuracy: {best_accuracy:.5f}')
                break

            print(f'[Round {n+1:04d}] Train_Accuracy: {accuracy:.5f}')

In [5]:
ada_boost = AdaBoost(tic_tac_toe_df)

In [6]:
ada_boost.train()

[Round 0001] Train_Accuracy: 0.69937
[Round 0002] Train_Accuracy: 0.69937
[Round 0003] Train_Accuracy: 0.63883
[Round 0004] Train_Accuracy: 0.70042
[Round 0005] Train_Accuracy: 0.74426
[Round 0006] Train_Accuracy: 0.69415
[Round 0007] Train_Accuracy: 0.68789
[Round 0008] Train_Accuracy: 0.72651
[Round 0009] Train_Accuracy: 0.74322
[Round 0010] Train_Accuracy: 0.75052
[Round 0011] Train_Accuracy: 0.76305
[Round 0012] Train_Accuracy: 0.76305
[Round 0013] Train_Accuracy: 0.78497
[Round 0014] Train_Accuracy: 0.78184
[Round 0015] Train_Accuracy: 0.79645
[Round 0016] Train_Accuracy: 0.77975
[Round 0017] Train_Accuracy: 0.80480
[Round 0018] Train_Accuracy: 0.80167
[Round 0019] Train_Accuracy: 0.82568
[Round 0020] Train_Accuracy: 0.83403
[Round 0021] Train_Accuracy: 0.91023
[Round 0022] Train_Accuracy: 0.86326
[Round 0023] Train_Accuracy: 0.85699
[Round 0024] Train_Accuracy: 0.83612
[Round 0025] Train_Accuracy: 0.83194
[Round 0026] Train_Accuracy: 0.84551
[Round 0027] Train_Accuracy: 0.85282
[